In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#讀入資料並判斷缺數值、非數字

In [2]:
org_data = pd.read_csv('HW3_hr-analytics.csv')

#前處理

In [3]:
OH_data = org_data.copy()

lin_i = [0,1,3]# 非離散型資料(col)

#去除OUTLIER
del_index = set()
for col in org_data.columns:
    
    if col in ['satisfaction_level','last_evaluation,average_montly_hours','time_spend_company']: 
        q1 = OH_data.describe()[col]['25%']
        q3 = OH_data.describe()[col]['75%']
        max_outlier = OH_data.describe()[col]['50%'] + (q3-q1)*3
        min_outlier = OH_data.describe()[col]['50%'] - (q3-q1)*3
        del_index = del_index.union(OH_data[(OH_data[col]>max_outlier) | (OH_data[col]<min_outlier)].index)
OH_data.drop(del_index,inplace=True,axis=0)
OH_data.reset_index(inplace=True,drop=True)


for i,col in enumerate(OH_data.columns):
    
    #將非離散型性資料等距離散
    if i == 0:
        OH_data[col] = pd.qcut(OH_data[col].rank(method='first'),4,labels=[1,2,3,4])
        matrix =np.array(OH_data[col])
        matrix = matrix[:, np.newaxis]
        continue
    
    #將非離散型性資料等距離散
    elif i in lin_i:
       OH_data[col] = pd.qcut(OH_data[col].rank(method='first'),4,labels=[1,2,3,4])
       
    #切割目標
    elif col == 'left':
        OH_temp = np.array(OH_data[col])
        target = OH_temp[:,np.newaxis]
        
    #OnehotEncoding       
    OH = OneHotEncoder()
    OH_temp = np.array(OH_data[col])
    OH_temp = OH_temp[:,np.newaxis]
    OH.fit(OH_temp)
    OH_temp = OH.transform(OH_temp).toarray()
    matrix = np.hstack((matrix,OH_temp))

#cut data     
train_data, test_data ,train_target , test_target = train_test_split(matrix,target,test_size=0.3,random_state=2019)  

#模型訓練

In [8]:
log = LogisticRegression()
log.fit(train_data,train_target.ravel())
test_pred = log.predict(test_data)

corr = np.sum(np.array(test_pred)[:,np.newaxis] == test_target)
length = len(test_data)
score = log.score(test_data, test_target)*100

#評分
print(f'accuracy : {corr}/{length}  {score:.2f}%')

accuracy : 4331/4331  100.00


總結:因為Logistic Regression 的特性，因此需要將資料離散化可以更好收斂，線性資料以等距切成4分，
再將全部資料使用Onehotencoding編碼進行訓練